In [1]:
import psycopg2
import pandas as pd

In [2]:
conn = psycopg2.connect(database='postgres', user='postgres',
                            password='postgres', host='10.11.6.13', port='54321')
cur = conn.cursor()

# remove the codes that cannot be parsed by javalang
syntax_error_ids = [3273863, 3867253, 4191191, 4191192, 4519111, 4519112, 5264639, 9532844, 9830264, 11714040,
                    13525465, 14507873, 16226644, 16532114, 17286897, 18347503, 22561089, 23117206]

In [3]:
select_pos_full = 'select function_id_one, function_id_two, functionality_id, syntactic_type, similarity_line, similarity_token from clones'
cur.execute(select_pos_full)
pos_full = cur.fetchall()
pos_full

[(17568209, 19141604, 4, 3, 0.0344827586206896, 0.146666666666667),
 (14164924, 19141604, 4, 3, 0.0526315789473685, 0.282051282051282),
 (19141604, 23069799, 4, 3, 0.0526315789473685, 0.295652173913043),
 (5035872, 19141604, 4, 3, 0.0357142857142857, 0.210191082802548),
 (120730, 19141604, 4, 3, 0.0120481927710844, 0.044973544973545),
 (3978711, 19141604, 4, 3, 0.8, 0.972222222222222),
 (916271, 19141604, 4, 3, 0.1, 0.126436781609195),
 (9791183, 19141604, 4, 3, 0.0769230769230769, 0.284403669724771),
 (13902980, 19141604, 4, 3, 0.0370370370370371, 0.170984455958549),
 (5875194, 19141604, 4, 3, 0.0454545454545454, 0.0949720670391061),
 (19141604, 19645662, 4, 3, 0.0476190476190477, 0.108552631578947),
 (13026685, 19141604, 4, 3, 0.0222222222222223, 0.065439672801636),
 (91017, 19141604, 4, 3, 0.0120481927710844, 0.044973544973545),
 (17454682, 19141604, 4, 3, 0.0172413793103449, 0.034020618556701),
 (8612647, 19141604, 4, 3, 0.0370370370370371, 0.170984455958549),
 (18779982, 19141604,

In [4]:
len(pos_full)

17168306

In [5]:
pos_full_set = list(set(pos_full))
len(pos_full_set)

8584153

In [6]:
pos_full_set_right = [
    pair
    for pair in pos_full_set
    if pair[0] not in syntax_error_ids and pair[1] not in syntax_error_ids
]
len(pos_full_set_right)

8578862

In [7]:
select_neg_full = 'select function_id_one, function_id_two, functionality_id, syntactic_type, similarity_line, similarity_token from false_positives'
cur.execute(select_neg_full)
neg_full = cur.fetchall()
neg_full

[(772874, 23677215, 39, 3, 0.011658, 0.0146144),
 (2061557, 23677215, 39, 3, 0.114286, 0.38565),
 (2141807, 23677215, 39, 3, 0.114286, 0.388889),
 (993834, 23677215, 39, 3, 0.0414201, 0.0680412),
 (1150035, 23677214, 39, 3, 0.0434783, 0.0451467),
 (2660645, 23677214, 39, 3, 0.125, 0.185),
 (1305322, 23677214, 39, 3, 0.333333, 0.3),
 (1008215, 23677214, 39, 3, 0.08, 0.181395),
 (1824930, 23677214, 39, 3, 0.166667, 0.217877),
 (3297899, 23677214, 39, 3, 0.0681818, 0.143357),
 (1442545, 23677214, 39, 3, 0.0483871, 0.0766129),
 (1784527, 23677214, 39, 3, 0.0833333, 0.121406),
 (2721355, 23677214, 39, 3, 0.0697674, 0.122257),
 (2084423, 23677214, 39, 3, 0.0714286, 0.10687),
 (67053, 23677214, 39, 3, 0.0163934, 0.019797),
 (1388070, 23677214, 39, 3, 0.106383, 0.119266),
 (2108369, 23677214, 39, 3, 0.363636, 0.553846),
 (2082745, 23677215, 39, 3, 0.037037, 0.418848),
 (3090992, 23677215, 39, 3, 0.046729, 0.0969697),
 (2906686, 23677215, 39, 3, 0.136364, 0.393701),
 (1181597, 23677215, 39, 3, 

In [8]:
len(neg_full)

558064

In [9]:
neg_full_set = list(set(neg_full))
len(neg_full_set)  

279032

In [10]:
neg_full_set_right = [
    pair
    for pair in neg_full_set
    if pair[0] not in syntax_error_ids and pair[1] not in syntax_error_ids
]
len(neg_full_set_right)


278957

In [11]:
# convert to label
id1 = []
id2 = []
label = []
func_id = []
for pair in pos_full_set_right:
    id1.append(pair[0])
    id2.append(pair[1])
    func_id.append(pair[2])
    if pair[3] == 3:
        similarity = min(pair[4], pair[5])
        if similarity < 0.5:
            label.append(5)
        elif similarity > 0.7:
            label.append(3)
        else:
            label.append(4)
    else:
        label.append(pair[3])

for pair in neg_full_set_right:
    id1.append(pair[0])
    id2.append(pair[1])
    label.append(0)
    func_id.append(pair[2])

bcb2015_pair = {'id1': id1, 'id2': id2, 'label': label, 'func_id': func_id}
bcb2015_pair_df = pd.DataFrame(data=bcb2015_pair)
print('-' * 50)
print('BCB 2015 example:')
print(bcb2015_pair_df.head())
print('Label distribution')
print(bcb2015_pair_df['label'].value_counts())

--------------------------------------------------
BCB 2015 example:
        id1       id2  label  func_id
0   4594844  17475530      5        4
1   2390396   7730595      5        2
2    772630   1753731      5       35
3  14318467  18202328      5        3
4   8364554  14729256      5        4
Label distribution
5    8418776
0     278957
4      87082
1      48116
3      20654
2       4234
Name: label, dtype: int64


In [12]:
neg_pair = bcb2015_pair_df[bcb2015_pair_df['label'].isin([0])]
neg_pair['func_id'].value_counts()

4     197334
2      38124
7      15162
14     11684
3       4499
6       4147
10      1762
9       1272
30       684
13       585
27       480
17       444
43       340
19       291
26       270
44       213
37       195
25       170
29       150
39       142
45       125
18       105
28       104
22       101
12        80
8         78
34        72
40        49
41        47
35        43
24        42
21        40
32        33
33        27
23        23
15        15
5         12
36         6
38         4
42         3
Name: func_id, dtype: int64

In [13]:
pos_pair = bcb2015_pair_df[~bcb2015_pair_df['label'].isin([0])]
pos_pair['func_id'].value_counts()

4     4661897
30     965347
3      890442
2      409058
35     350690
10     240470
42     124747
41     119803
34     106489
31     104099
27      83434
14      81003
23      78606
24      67461
39      37674
33      37396
32      28680
6       24530
20      22366
36      21097
40      20301
28      19306
44      14028
7       13861
45      12561
37      11780
43      10877
19       5251
25       5137
13       3081
22       2775
29       1711
18        736
15        325
21        299
8         276
26        253
38        253
11        252
12        210
17        210
9          55
5          35
Name: func_id, dtype: int64

In [14]:
bcb2015_pair_df['func_id'].value_counts()

4     4859231
30     966031
3      894941
2      447182
35     350733
10     242232
42     124750
41     119850
34     106561
31     104099
14      92687
27      83914
23      78629
24      67503
39      37816
33      37423
7       29023
32      28713
6       28677
20      22366
36      21103
40      20350
28      19410
44      14241
45      12686
37      11975
43      11217
19       5542
25       5307
13       3666
22       2876
29       1861
9        1327
18        841
17        654
26        523
8         354
15        340
21        339
12        290
38        257
11        252
5          47
Name: func_id, dtype: int64

In [15]:
train_df = bcb2015_pair_df[~bcb2015_pair_df['func_id'].isin(
                                                            [7, 6, 30, 17, 26, 25, 45, 22, 34, 35, 14, 10, 13, 43, 44, 29, 18, 12, 40, 24])]
train_df['label'].value_counts()                                                            

5    1919139
0      45395
4      44378
1      20624
3      10191
2        902
Name: label, dtype: int64

In [16]:
valid_df = bcb2015_pair_df[bcb2015_pair_df['func_id'].isin(
        [7, 6, 30, 17, 26, 25, 45, 22, 34, 35, 14])]
valid_df['label'].value_counts() 

5    1468789
0      21218
4       8810
3       2155
1       1989
2        110
Name: label, dtype: int64

In [17]:
test_df = bcb2015_pair_df[bcb2015_pair_df['func_id'].isin(
        [])]
test_df['label'].value_counts() 

5    415002
0     15010
1     11701
4     10206
3      2863
2       106
Name: label, dtype: int64

In [18]:
train_neg_label = train_df[(train_df['label'] == 0)]
train_pos_label = train_df[~(train_df['label'] == 0)].sample(len(train_neg_label), random_state=555)
train_label = pd.concat([train_pos_label, train_neg_label], axis=0)
train_label = train_label.sample(frac=1, random_state=555).drop(['func_id'], axis=1).reset_index(drop=True)
train_label

,id1,id2,label
0,18517215,23677111,0
1,20108469,23677150,0
2,9689181,13024538,5
3,1481849,23677112,0
4,113869,278371,5
...,...,...,...
90785,3732741,8162671,5
90786,5318668,6096495,0
90787,10841569,20603727,4
90788,8258096,17994731,5


In [19]:
valid_neg_label = valid_df[(valid_df['label'] == 0)]
valid_pos_label = valid_df[~(valid_df['label'] == 0)].sample(len(valid_neg_label), random_state=555)
valid_label = pd.concat([valid_pos_label, valid_neg_label], axis=0)
valid_label = valid_label.sample(frac=1, random_state=555).drop(['func_id'], axis=1).reset_index(drop=True)
valid_label

,id1,id2,label
0,2194576,2800377,5
1,122369,23677144,0
2,5404386,21106542,5
3,2539368,17698952,5
4,6189133,10137489,5
...,...,...,...
42431,915504,23677133,0
42432,2053517,23677136,0
42433,1611680,23677141,0
42434,1196912,9781390,5


In [20]:
test_neg_label = test_df[(test_df['label'] == 0)]
test_pos_label = test_df[~(test_df['label'] == 0)].sample(len(test_neg_label), random_state=555)
test_label = pd.concat([test_pos_label, test_neg_label], axis=0)
test_label = test_label.sample(frac=1, random_state=555).drop(['func_id'], axis=1).reset_index(drop=True)
test_label

,id1,id2,label
0,1730523,21664404,5
1,4628439,17116115,5
2,8355906,22443179,5
3,9662694,23677160,0
4,14256887,23677160,0
...,...,...,...
30015,8561731,11840277,5
30016,14970465,23677159,0
30017,18818132,23677160,0
30018,160206,23677226,0


In [21]:
train_str = ''
for i in range(len(train_label)):
    train_str += str(train_label['id1'][i])
    train_str += '\t'
    train_str += str(train_label['id2'][i])
    train_str += '\t'
    train_str += '0' if train_label['label'][i] == 0 else '1'
    train_str += '\n'

with open('function_out_split/train.txt', 'w', encoding='utf-8') as f:
    f.write(train_str)

In [22]:
valid_str = ''
for i in range(len(valid_label)):
    valid_str += str(valid_label['id1'][i])
    valid_str += '\t'
    valid_str += str(valid_label['id2'][i])
    valid_str += '\t'
    valid_str += '0' if valid_label['label'][i] == 0 else '1'
    valid_str += '\n'

with open('function_out_split/valid.txt', 'w', encoding='utf-8') as f:
    f.write(valid_str)

In [23]:
test_str = ''
for i in range(len(test_label)):
    test_str += str(test_label['id1'][i])
    test_str += '\t'
    test_str += str(test_label['id2'][i])
    test_str += '\t'
    test_str += '0' if test_label['label'][i] == 0 else '1'
    test_str += '\n'

with open('function_out_split/test.txt', 'w', encoding='utf-8') as f:
    f.write(test_str)

In [24]:
bcb_func_all = pd.read_csv('../bcb2015_full_funcs_all.csv', header=0, encoding='utf-8')
bcb_func_all

,id,code
0,74,"static void copy (String src, String dest) thr..."
1,525,ResultHelperTests () throws Exception {\n N...
2,587,"public HTMLDocument handleURL (String suburl, ..."
3,661,"public void convert (File src, File dest) thro..."
4,778,public static void main (String [] args) {\n ...
...,...,...
73296,23677221,public static List < String > extractMatches (...
73297,23677222,"public static void copyDirectory1 (Path src, P..."
73298,23677223,public static void copyDirectory2 (final Path ...
73299,23677226,public static boolean isPalindrome (String ori...


In [25]:
bcb_func_all = bcb_func_all.rename(columns={'code': 'func', 'id': 'idx'})
bcb_func_all.to_json(path_or_buf='function_out_split/data.jsonl',
                     orient='records', lines=True)